# Task
Realizar o ajuste fino de um modelo DeepLabV3 para detecção de culturas e ervas daninhas usando o dataset disponível em "https://www.kaggle.com/datasets/ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes".

## Download e extração dos dados
Baixar o dataset do Kaggle e extraí-lo para um diretório local.


In [ ]:
import os
kaggle_cred_path = '/content/kaggle.json'

if os.path.exists(kaggle_cred_path):
    # Define a variável de ambiente para que o Kaggle encontre o arquivo
    os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(kaggle_cred_path)

    # Define as permissões corretas (somente o proprietário pode ler/escrever)
    !chmod 600 {kaggle_cred_path}

    print(f"Credenciais do Kaggle carregadas com sucesso de {kaggle_cred_path}")
else:
    print(f"Erro: Arquivo 'kaggle.json' não encontrado em {kaggle_cred_path}")
    print("Por favor, faça o upload do seu 'kaggle.json' para o diretório /content/ (raiz).")

In [ ]:
import os
import zipfile

# Create a directory for the dataset
dataset_dir = 'crop_weed_dataset'
os.makedirs(dataset_dir, exist_ok=True)

# Download the dataset using Kaggle API
dataset_name = 'ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes'
download_command = f'kaggle datasets download -d {dataset_name} -p {dataset_dir}'
print(f"Executing download command: {download_command}")
os.system(download_command)

# Unzip the dataset
zip_file_path = os.path.join(dataset_dir, os.path.basename(dataset_name).replace('/', '_') + '.zip')
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

print(f"Dataset downloaded and extracted to {dataset_dir}")

## Preparação dos dados
Carregar as imagens e as anotações (bounding boxes) e prepará-las para o treinamento do modelo DeepLabV3. Isso pode incluir redimensionamento, normalização e criação de máscaras de segmentação a partir das bounding boxes.

In [ ]:
import cv2
import numpy as np
from PIL import Image


data_dir = 'crop_weed_dataset/agri_data/data'
mask_dir = 'data/masks'

# IMPORTANTE: Para nossas máscaras, queremos 0:fundo, 1:crop, 2:weed.
class_map = {
    0: 1,  # crop
    1: 2   # weed
}
# --------------------------------
os.makedirs(mask_dir, exist_ok=True)

print("Iniciando criação de máscaras a partir de anotações .txt (YOLO)...")

# Iterar sobre todos os arquivos de anotação .txt
for txt_file in os.listdir(data_dir):
    if not txt_file.endswith('.txt'):
        continue

    # Encontrar a imagem correspondente
    img_filename = txt_file.replace('.txt', '.jpeg')
    img_path = os.path.join(data_dir, img_filename)

    if not os.path.exists(img_path):
        print(f"Aviso: Imagem não encontrada para a anotação '{txt_file}'")
        continue

    # Obter as dimensões da imagem
    try:
        # Usar OpenCV para ler as dimensões
        image = cv2.imread(img_path)
        if image is None:
            raise Exception(f"Não foi possível ler a imagem {img_path}")
        height, width, _ = image.shape
    except Exception as e:
        print(f"Erro ao ler imagem {img_path}: {e}")
        continue

    # Criar uma máscara em branco (0s) com as dimensões da imagem
    mask = np.zeros((height, width), dtype=np.uint8)

    # Abrir o arquivo .txt de anotação
    with open(os.path.join(data_dir, txt_file), 'r') as f:
        for line in f.readlines():
            try:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center_norm = float(parts[1])
                y_center_norm = float(parts[2])
                width_norm = float(parts[3])
                height_norm = float(parts[4])

                # Obter o valor do pixel para a máscara (1 para crop, 2 para weed)
                pixel_value = class_map.get(class_id)
                if pixel_value is None:
                    continue # Ignora classes que não estão no nosso map

                # Desnormalizar as coordenadas
                box_width = width_norm * width
                box_height = height_norm * height
                x_center = x_center_norm * width
                y_center = y_center_norm * height

                # Converter para coordenadas (xmin, ymin, xmax, ymax)
                xmin = int(x_center - (box_width / 2))
                ymin = int(y_center - (box_height / 2))
                xmax = int(x_center + (box_width / 2))
                ymax = int(y_center + (box_height / 2))

                # Garantir que as coordenadas estejam dentro dos limites da imagem
                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(width, xmax)
                ymax = min(height, ymax)

                # Desenhar o retângulo preenchido na máscara
                # cv2.rectangle(imagem, (xmin, ymin), (xmax, ymax), cor, preenchimento)
                cv2.rectangle(mask, (xmin, ymin), (xmax, ymax),
                              color=int(pixel_value), thickness=cv2.FILLED)

            except Exception as e:
                print(f"Erro ao processar linha '{line}' no arquivo {txt_file}: {e}")

    # Salvar a máscara como um arquivo .png
    mask_pil = Image.fromarray(mask)
    mask_filename = img_filename.replace('.jpeg', '.png')
    mask_pil.save(os.path.join(mask_dir, mask_filename))

print(f"\nProcessamento concluído. Máscaras salvas em {mask_dir}")


##Dividir os dados em conjuntos de treinamento e validação.

* Listar Arquivos: Criar uma lista de todos os nomes de arquivo base (ex: ['agri_0_1002', 'agri_0_1007', ...]).

* Dividir a Lista: Usar sklearn.model_selection.train_test_split para dividir essa lista de nomes.

In [ ]:

import shutil
from sklearn.model_selection import train_test_split

# Imagens originais estão aqui:
image_dir = data_dir

# dataset final organizado:
output_dataset_dir = 'dataset_preparado'
# ---------------------------------

# Criar a estrutura de pastas de destino
train_img_path = os.path.join(output_dataset_dir, 'images', 'train')
val_img_path = os.path.join(output_dataset_dir, 'images', 'val')
train_mask_path = os.path.join(output_dataset_dir, 'masks', 'train')
val_mask_path = os.path.join(output_dataset_dir, 'masks', 'val')

os.makedirs(train_img_path, exist_ok=True)
os.makedirs(val_img_path, exist_ok=True)
os.makedirs(train_mask_path, exist_ok=True)
os.makedirs(val_mask_path, exist_ok=True)


# 1. Listar arquivos (vamos usar as imagens .jpeg como referência)
try:
    image_files_jpeg = [f for f in os.listdir(image_dir) if f.endswith('.jpeg')]
    # Obter os nomes base (sem extensão)
    file_bases = [f.replace('.jpeg', '') for f in image_files_jpeg]
except FileNotFoundError:
    print(f"ERRO: Não encontrei o diretório de imagens em '{image_dir}'")
    raise

if not file_bases:
    print(f"ERRO: Nenhuma imagem .jpeg encontrada em '{image_dir}'")
    raise

# 2. Dividir a lista de nomes
train_files, val_files = train_test_split(file_bases, test_size=0.2, random_state=42)

print(f"Total de pares (imagem/máscara): {len(file_bases)}")
print(f"Conjunto de treino: {len(train_files)}")
print(f"Conjunto de validação: {len(val_files)}")

# 3. Função auxiliar para COPIAR os arquivos
def copy_files(file_list, source_img_dir, source_mask_dir, dest_img_dir, dest_mask_dir):
    count = 0
    missing_masks = 0
    for file_base in file_list:
        img_src = os.path.join(source_img_dir, file_base + '.jpeg')
        mask_src = os.path.join(source_mask_dir, file_base + '.png') # Assumindo máscara .png

        # Verificar se ambos os arquivos existem antes de copiar
        if os.path.exists(img_src) and os.path.exists(mask_src):
            shutil.copy(img_src, os.path.join(dest_img_dir, file_base + '.jpeg'))
            shutil.copy(mask_src, os.path.join(dest_mask_dir, file_base + '.png'))
            count += 1
        elif not os.path.exists(mask_src):
            print(f"Aviso: Máscara não encontrada para '{file_base}.png' em {source_mask_dir}")
            missing_masks += 1
        else:
            print(f"Aviso: Imagem não encontrada para '{file_base}.jpeg' em {source_img_dir}")
    return count, missing_masks

# 4. Copiar os arquivos para a nova estrutura
print("Copiando arquivos de treino...")
num_train, missing_train = copy_files(train_files, image_dir, mask_dir, train_img_path, train_mask_path)

print("Copiando arquivos de validação...")
num_val, missing_val = copy_files(val_files, image_dir, mask_dir, val_img_path, val_mask_path)

print("\n--- Concluído! ---")
print(f"{num_train} pares de treino copiados para '{output_dataset_dir}/'")
print(f"{num_val} pares de validação copiados para '{output_dataset_dir}/'")
if (missing_train + missing_val) > 0:
    print(f"ATENÇÃO: {missing_train + missing_val} máscaras não foram encontradas e seus pares foram ignorados.")

print("\nSua nova estrutura de dataset está pronta em 'dataset_preparado/'.")

##Data Augmentation
Para segmentação, é crucial que transformações espaciais (como rotação ou flip) sejam aplicadas exatamente da mesma forma na imagem e na sua máscara. A biblioteca albumentations faz isso automaticamente.

* Para Treino: Apliquei várias aumentações para tornar o modelo robusto.

* Para Validação: Apliquei apenas o pré-processamento básico
(redimensionamento e normalização) para obter uma avaliação consistente.

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Definir um tamanho padrão para o qual todas as imagens serão redimensionadas
IMG_SIZE = 256

# Definir as médias e desvios padrão do ImageNet (usados para pré-treinamento)
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

# 1. Pipeline de transformações para o conjunto de TREINO
train_transform = A.Compose([
    # Redimensiona mantendo o aspecto e preenchendo (Pad) ou cortando (RandomCrop)
    A.LongestMaxSize(max_size=IMG_SIZE),
    A.PadIfNeeded(min_height=IMG_SIZE, min_width=IMG_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),

    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),

    # Aumentações de cor (aplicadas SOMENTE à imagem)
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.3),

    # Normalização e conversão para Tensor
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(), # Converte imagem e máscara para Tensores PyTorch
])

# 2. Pipeline de transformações para o conjunto de VALIDAÇÃO
val_transform = A.Compose([
    # Apenas redimensiona para o tamanho esperado pelo modelo
    A.LongestMaxSize(max_size=IMG_SIZE),
    A.PadIfNeeded(min_height=IMG_SIZE, min_width=IMG_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),

    # Normalização e conversão para Tensor
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

##Criação do Dataset e DataLoaders (PyTorch)
Agora, criarei uma classe Dataset personalizada que o PyTorch pode usar. Ela irá:

1. Encontrar os pares de imagem/máscara.

2. Carregá-los do disco.

3. Aplicar as transformações que definimos acima.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CropWeedDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        # Lista apenas os arquivos de imagem
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.jpeg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Nome do arquivo de imagem
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)

        # Nome do arquivo de máscara correspondente
        mask_name = img_name.replace('.jpeg', '.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Carregar imagem (OpenCV lê em BGR, convertemos para RGB)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Carregar máscara (em tons de cinza, exatamente como foi salva)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Nossas classes na máscara são 0 (fundo), 1 (crop), 2 (weed)

        # Aplicar transformações
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

            # A função de perda (CrossEntropyLoss) espera que a máscara
            # seja do tipo Long (inteiro de 64 bits), não Float.
            mask = mask.long()

        return image, mask

# --- Caminhos para os dados preparados ---
# fiz outras variaveis pra ter certeza
TRAIN_IMG_DIR = 'dataset_preparado/images/train'
TRAIN_MASK_DIR = 'dataset_preparado/masks/train'
VAL_IMG_DIR = 'dataset_preparado/images/val'
VAL_MASK_DIR = 'dataset_preparado/masks/val'

# 1. Instanciar os Datasets
train_dataset = CropWeedDataset(
    image_dir=TRAIN_IMG_DIR,
    mask_dir=TRAIN_MASK_DIR,
    transform=train_transform
)

val_dataset = CropWeedDataset(
    image_dir=VAL_IMG_DIR,
    mask_dir=VAL_MASK_DIR,
    transform=val_transform
)

# 2. Instanciar os DataLoaders
BATCH_SIZE = 8 # Ajuste conforme a memória da sua GPU

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # Embaralhar dados de treino é crucial
    num_workers=4, # Use múltiplos processos para carregar dados
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # Não há necessidade de embaralhar a validação
    num_workers=4,
    pin_memory=True
)

print(f"DataLoaders prontos!")
print(f"Lotes de Treino ({len(train_dataset)} imagens): {len(train_loader)} lotes de tamanho {BATCH_SIZE}")
print(f"Lotes de Validação ({len(val_dataset)} imagens): {len(val_loader)} lotes de tamanho {BATCH_SIZE}")

##Carregar o Modelo DeepLabv3
Vamos usar um modelo DeepLabv3 pré-treinado (com backbone ResNet-50) do torchvision e adaptá-lo para o nosso problema. O modelo original foi treinado no dataset COCO com 21 classes. Precisamos trocar a "cabeça" (camada de classificação final) para que ela produza saídas para as nossas 3 classes:

* 0: Background

* 1: Crop

* 2: Weed

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights

# --- Configurações Iniciais ---
NUM_CLASSES = 3  # (0: Fundo, 1: Crop, 2: Weed)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
NUM_EPOCHS = 2  # Comece com 10-20 e ajuste conforme necessário

print(f"Usando dispositivo: {DEVICE}")

# 1. Carregar o modelo DeepLabv3 pré-treinado
# Usamos os pesos mais recentes (Weights.COCO_WITH_VOC_LABELS_V1)
weights = DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
model = deeplabv3_resnet50(weights=weights)

"""
2. Modificar a camada de classificação final (classifier)
-> O DeepLabv3 no torchvision usa um 'classifier' com 5 camadas
-> A última camada (índice 4) é um Conv2d que projeta para 21 classes (COCO)
-> Vamos substituí-la por uma nova Conv2d para nossas 3 classes.
"""

# Obter o número de canais de entrada da camada antiga
in_channels = model.classifier[4].in_channels

# Criar a nova camada de classificação
model.classifier[4] = nn.Conv2d(
    in_channels,
    NUM_CLASSES,
    kernel_size=1, # Kernel 1x1 é suficiente para a classificação final
    stride=1
)

# 3. Mover o modelo para o dispositivo (GPU ou CPU)
model = model.to(DEVICE)

print("Modelo DeepLabv3 carregado e modificado para 3 classes.")

##Definir Função de Perda, Otimizador e Loop de Treinamento
Agora, definimos como o modelo aprenderá.

* Função de Perda (Loss): nn.CrossEntropyLoss. É a escolha padrão para segmentação multiclasse. Ela compara os logits (saída bruta do modelo) com as máscaras de inteiros (0, 1, 2).

* Otimizador: torch.optim.Adam. Um otimizador robusto e popular.

In [ ]:
# Definir Perda e Otimizador
# CrossEntropyLoss é ideal, pois nossas máscaras são 0, 1, 2
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

'''
 (Opcional) Um scheduler para ajustar a taxa de aprendizado
 scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)
'''

print("Função de perda (CrossEntropy) e Otimizador (Adam) definidos.")

##O Loop de Treinamento e Validação
Este é o coração do processo. Vamos criar uma função para uma época de treino (train_fn) e uma para validação (val_fn).

Nota Importante sobre a Saída do DeepLab: O modelo deeplabv3 do torchvision retorna um dicionário. A saída de segmentação principal está na chave 'out'.

In [ ]:
from tqdm import tqdm

def train_fn(loader, model, optimizer, loss_fn):
    """Executa uma época de treinamento."""
    loop = tqdm(loader, desc="Treinando")

    total_loss = 0.0
    model.train() # Coloca o modelo em modo de treinamento

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE) # targets já são (N, H, W) e tipo Long

        # 1. Forward pass
        # A saída do deeplab é um dicionário, pegamos a chave 'out'
        predictions = model(data)['out'] # Saída é (N, C=3, H, W)

        # 2. Calcular a perda
        loss = loss_fn(predictions, targets)

        # 3. Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Atualiza a barra de progresso
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    print(f"-> Fim da Época de Treino. Perda Média: {avg_loss:.4f}")
    return avg_loss

def val_fn(loader, model, loss_fn):
    """Executa uma época de validação."""
    loop = tqdm(loader, desc="Validando", leave=False) # 'leave=False' para limpar após terminar

    model.eval() # Coloca o modelo em modo de avaliação (desliga dropout, etc.)
    total_val_loss = 0.0
    num_correct = 0
    num_pixels = 0

    with torch.no_grad(): # Desliga o cálculo de gradientes
        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device=DEVICE)
            targets = targets.to(device=DEVICE)

            # Forward pass
            predictions = model(data)['out']

            # Calcular perda
            loss = loss_fn(predictions, targets)
            total_val_loss += loss.item()

            # Calcular Acurácia de Pixel
            # Pega o índice da classe com maior logit (canal 0, 1 ou 2)
            preds_labels = predictions.argmax(dim=1) # (N, H, W)
            num_correct += (preds_labels == targets).sum()
            num_pixels += torch.numel(targets) # Total de pixels (N * H * W)

    avg_loss = total_val_loss / len(loader)
    pixel_accuracy = (num_correct / num_pixels) * 100

    print(f"-> Fim da Validação. Perda Média: {avg_loss:.4f}, Acurácia de Pixel: {pixel_accuracy:.2f}%")
    return avg_loss, pixel_accuracy


# --- O Loop Principal de Treinamento ---

print("\n--- Iniciando o Treinamento ---")

# Variáveis para salvar o melhor modelo
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    print(f"\nÉpoca [{epoch+1}/{NUM_EPOCHS}]")

    train_loss = train_fn(train_loader, model, optimizer, loss_fn)
    val_loss, val_accuracy = val_fn(val_loader, model, loss_fn)

    """
    (Opcional) Atualizar o scheduler
    scheduler.step(val_loss)
    """

    # Salvar o melhor modelo (baseado na menor perda de validação)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_deeplab_model.pth")
        print(f"Modelo salvo! Nova melhor perda de validação: {best_val_loss:.4f}")

print("\n--- Treinamento Concluído! ---")
print(f"O melhor modelo foi salvo em 'best_deeplab_model.pth'")

## Avaliação Visual e Inferência do Modelo

Ter um modelo treinado e salvo é ótimo, mas como ele realmente se parece?

Esta célula final de teste visual nos permite avaliar o desempenho qualitativo do modelo.

1.  **Recriar a Arquitetura:** O script primeiro reconstrói a arquitetura do `deeplabv3_resnet50` e modifica a camada final para 3 classes, exatamente como fizemos antes do treinamento.
2.  **Carregar Pesos:** Ele carrega os pesos do melhor modelo salvo (`best_deeplab_model.pth`).
3.  **Modo de Avaliação:** O modelo é colocado em modo de inferência (`model.eval()`).
4.  **Selecionar Amostras:** Ele pega algumas imagens aleatórias do `val_dataset` (o conjunto de validação que o modelo não viu durante o treino).
5.  **Fazer a Predição:** Para cada imagem, ele:
    * Envia a imagem para o modelo.
    * Pega a saída `['out']` (logits).
    * Usa `argmax(dim=1)` para encontrar a classe (0, 1, ou 2) com a maior pontuação para cada pixel.
6.  **Plotar Comparação:** Por fim, ele exibe uma comparação lado a lado da **Imagem Original**, da **Máscara Real (Ground Truth)** e da **Máscara Prevista** pelo modelo.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torchvision.models.segmentation import deeplabv3_resnet50
import random

# --- 1. Recarregar Configurações Essenciais ---
# Estas variáveis devem corresponder às células anteriores do seu notebook

# (Da Célula 13)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 3  # (0: Fundo, 1: Crop, 2: Weed)

# (Da Célula 17)
MODEL_PATH = "best_deeplab_model.pth"

# (Da Célula 10) - Usado para reverter a normalização para visualização
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

# --- 2. Recarregar a arquitetura do modelo ---
# Deve-se recriar a arquitetura EXATAMENTE como no treinamento
print("Recriando a arquitetura do modelo...")
model = deeplabv3_resnet50(weights=None) # Os pesos serão inseridos, por isso None

# Modificar a cabeça para corresponder ao NUM_CLASSES (da célula 13)
in_channels = model.classifier[4].in_channels
model.classifier[4] = nn.Conv2d(
    in_channels,
    NUM_CLASSES,
    kernel_size=1,
    stride=1
)

# --- 3. Carregar os pesos treinados ---
try:
    print(f"Carregando pesos treinados de {MODEL_PATH}...")
    # Garante que o modelo seja carregado no dispositivo correto (CPU ou GPU)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
except FileNotFoundError:
    print(f"ERRO: Arquivo do modelo '{MODEL_PATH}' não encontrado.")
    print("Certifique-se de que o treinamento foi concluído e o arquivo foi salvo.")
except Exception as e:
    print(f"Erro inesperado ao carregar o modelo: {e}")

model.to(DEVICE)
model.eval()
print("Modelo pronto para inferência.")

# --- 4. Função para desnormalizar imagem (para visualização) ---
def denormalize(tensor_image):
    """Reverte a normalização (A.Normalize) para visualização com Matplotlib."""
    # (C, H, W) -> (H, W, C)
    img = tensor_image.cpu().permute(1, 2, 0).numpy()

    img = (img * STD) + MEAN
    # Garante que os valores fiquem entre [0, 1]
    img = np.clip(img, 0, 1)
    return img

# --- 5. Teste Visual no Conjunto de Validação ---

# Verifica se o 'val_dataset' (criado na célula 11) existe
if 'val_dataset' not in globals():
     print("ERRO: 'val_dataset' não está definido.")
     print("Por favor, execute a célula 11 ('Criação do Dataset e DataLoaders') primeiro.")
else:
    num_samples_to_show = 3 # Quantas imagens de teste você quer ver
    # Pega X índices aleatórios do dataset de validação
    indices = random.sample(range(len(val_dataset)), num_samples_to_show)

    print(f"\nMostrando {num_samples_to_show} predições aleatórias do conjunto de validação...")

    for idx in indices:
        # O 'val_dataset' já aplica as transformações (redimensionar, normalizar)
        image_tensor, gt_mask_tensor = val_dataset[idx]

        # Preparar o tensor da imagem para o modelo
        # Adiciona uma dimensão de "lote" (batch) [N, C, H, W]
        input_tensor = image_tensor.unsqueeze(0).to(DEVICE) # (1, 3, 256, 256)

        # --- 6. Executar Inferência ---
        with torch.no_grad(): # Desliga o cálculo de gradientes
            # A saída do deeplab é um dicionário
            output = model(input_tensor)['out']

        # --- 7. Pós-processar a saída ---
        # output é (1, 3, H, W)
        # .argmax(dim=1) pega o índice da classe com maior valor (0, 1 ou 2)
        # para cada pixel, resultando em (1, H, W)
        # .squeeze(0) remove a dimensão do lote -> (H, W)
        pred_mask = output.argmax(dim=1).squeeze(0).cpu().numpy()

        # Pegar a máscara real (ground truth)
        gt_mask = gt_mask_tensor.cpu().numpy() # (H, W)

        # Desnormalizar a imagem original para visualização
        original_image_vis = denormalize(image_tensor)

        # --- 8. Plotar os resultados ---
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f"Amostra de Validação (Índice: {idx})", fontsize=16)

        ax1.imshow(original_image_vis)
        ax1.set_title("Imagem de Entrada (Processada)")
        ax1.axis('off')

        # Usar vmin/vmax para garantir que as cores (0, 1, 2) sejam consistentes
        ax2.imshow(gt_mask, vmin=0, vmax=NUM_CLASSES-1)
        ax2.set_title("Máscara Real (Ground Truth)")
        ax2.axis('off')

        ax3.imshow(pred_mask, vmin=0, vmax=NUM_CLASSES-1)
        ax3.set_title("Máscara Prevista (Modelo)")
        ax3.axis('off')

        plt.show()